In [ ]:
pip install forex_python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.9/137.9 kB 3.1 MB/s eta 0:00:00


In [ ]:
import math
import yfinance as yf
import numpy as np
import pandas as pd
from datetime import datetime as datetime
from pandas.core.arrays.period import timedelta
from forex_python.converter import CurrencyRates
import random
from scipy.optimize import minimize

In [ ]:
def getDf():
  df = pd.read_csv('/content/etfs_list.csv')
  adj_close = []
  for ticker in df['symbol']:
    try:
      data = yf.download(ticker, progress=False, show_errors=False)
      adj_close.append(data['Adj Close'][-1])
    except:
      adj_close.append(math.inf)
  df["adj_close"] = adj_close
  return df
df = getDf()

yfinance: download(show_errors=False) argument is deprecated and will be removed in future version. Do this instead to suppress error messages: logging.getLogger('yfinance').setLevel(logging.CRITICAL)


In [ ]:
def getNumberOfEtf(amount):
  num_portfolios = {
    (1, 5000):5,
    (5001, 20000):10,
    (20000, math.inf): 15
  }
  no_etfs = 0
  for k in num_portfolios:
    if amount>=k[0] and amount<=k[1]:
      no_etfs = num_portfolios[k]
  return no_etfs

In [ ]:
def getPortfolios(number, etfs):
  if len(etfs) <= number:
    return [etfs]
  portfolios = []
  for i in range(10):
    portfolio = random.sample(etfs, number)
    portfolios.append(portfolio)
  return portfolios

In [ ]:
def adj_close(ticker, start, end):
  data = yf.download(ticker, progress=False, start=start, end=end)
  if data.shape[0] == 0:
    data = yf.download(ticker, progress=False)
  return data['Adj Close']

In [ ]:
# adj_close(['QQQ'], None, None)

In [ ]:
def standard_deviation(weights, cov_matrix):
    variance = weights.T @ cov_matrix @ weights
    return np.sqrt(variance)

def expected_return(weights, log_returns):
    return np.sum(log_returns.mean()*weights)*252

def sharpe_ratio(weights, log_returns, cov_matrix, risk_free_rate):
    return (expected_return(weights, log_returns) - risk_free_rate) / standard_deviation(weights, cov_matrix)
def neg_sharpe_ratio(weights, log_returns, cov_matrix, risk_free_rate):
    return -sharpe_ratio(weights, log_returns, cov_matrix, risk_free_rate)

In [ ]:
def optimized_portfolio(tickers, log_returns, cov_matrix, risk_free_rate):
  constraints = {'type': 'eq', 'fun': lambda weights: np.sum(weights) - 1}
  bounds = [(0, 0.4) for _ in range(len(tickers))]
  initial_weights = np.array([1/len(tickers)]*len(tickers))
  optimized_results = minimize(neg_sharpe_ratio, initial_weights, args=(log_returns, cov_matrix, risk_free_rate), method='SLSQP', constraints=constraints, bounds=bounds)
  optimal_weights = optimized_results.x
  result = {}
  result["weights"] = optimal_weights
  result["return"] = expected_return(optimal_weights, log_returns)
  result["volatility"] = standard_deviation(optimal_weights, cov_matrix)
  result["sharpRatio"] = sharpe_ratio(optimal_weights, log_returns, cov_matrix, risk_free_rate)
  return result

In [ ]:
def sharpRatioAndVaR(amount, years, etfs):
  end = datetime.today()
  start = end - timedelta(days=(years*1.5)*365)
  #calculate adj close
  adj_close_df = pd.DataFrame()
  for ticker in etfs:
    adj_close_df[ticker] = adj_close(ticker, start, end)
  adj_close_df.dropna(axis=0, inplace=True)

  #calculate log returns
  log_returns = np.log(adj_close_df/adj_close_df.shift(1))
  log_returns.dropna()

  #calculate covariance matrix
  cov_matrix = log_returns.cov() * 252

  #optimize portfolio
  risk_free_rate = 0.02
  result = optimized_portfolio(etfs, log_returns, cov_matrix, risk_free_rate)
  historical_returns = (log_returns * result['weights']).sum(axis =1)
  days = 5
  range_returns = historical_returns.rolling(window = days).sum()
  # print("before " , range_returns)
  range_returns = range_returns.dropna()
  confidence_interval = 0.99
  # print(historical_returns)
  # print(range_returns)

  VaR = -np.percentile(range_returns, 100 - (confidence_interval * 100))*amount
  result['VaR'] = VaR
  result['etfs'] = etfs
  # print(result)
  return result

In [ ]:
# sharpRatioAndVaR(3000, 10, ['SPY','BND','GLD','QQQ','VTI'])

In [ ]:
def getOptPortfolio(amount, years, risk_level):
  cr = CurrencyRates()
  amount = cr.convert('GBP', 'USD', amount)
  no_etfs = getNumberOfEtf(amount)
  data = df.loc[df["adj_close"]<=amount]
  etfs = list(data['symbol'])
  portfolios = getPortfolios(no_etfs, etfs)
  sharpRatioAndVaRs = []
  for portfolio in portfolios:
    try:
      sharpRatioAndVaRs.append(sharpRatioAndVaR(amount, years, portfolio))
    except:
      pass
  risk_amount = risk_level * amount
  sharpRatioAndVaRs = [x for x in sharpRatioAndVaRs if x["VaR"] <= risk_amount]
  if len(sharpRatioAndVaRs) == 0:
    sharpRatioAndVaRs.sort(key=lambda portfolio:portfolio['VaR'])
    return sharpRatioAndVaRs[0]
  sharpRatioAndVaRs.sort(key=lambda portfolio:portfolio['sharpRatio'])
  return sharpRatioAndVaRs[-1]

In [ ]:
r = getOptPortfolio(10000, 10, 0.5)

/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:518: RuntimeWarning: Mean of empty slice.
  avg = a.mean(axis, **keepdims_kw)
/usr/local/lib/python3.10/dist-packages/numpy/core/_methods.py:182: RuntimeWarning: invalid value encountered in divide
  ret = um.true_divide(
/usr/local/lib/python3.10/dist-packages/pandas/core/frame.py:10474: RuntimeWarning: Degrees of freedom <= 0 for slice
  base_cov = np.cov(mat.T, ddof=ddof)
/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:2704: RuntimeWarning: divide by zero encountered in divide
  c *= np.true_divide(1, fact)
/usr/local/lib/python3.10/dist-packages/numpy/lib/function_base.py:2704: RuntimeWarning: invalid value encountered in multiply
  c *= np.true_divide(1, fact)


In [ ]:
r

{'weights': array([0.4       , 0.        , 0.        , 0.        , 0.32184572,
        0.        , 0.        , 0.25217252, 0.        , 0.02598176]),
 'return': 0.3768080418780494,
 'volatility': 0.20567931042167012,
 'sharpRatio': 1.7347784818343912,
 'VaR': 974.0706556840678,
 'etfs': ['SMIN',
  'PHO',
  'BRND',
  'XLE',
  'GLIN',
  'FILL',
  'CRUD',
  'VAW',
  'ISHG',
  'HAUZ']}